# Line fitting using least squares

This is to do a first decomposition of the spectra present in the HC$_3$N (10-9) data cube.

In [1]:
import sys
sys.path.append('../')
from config import *
from astropy.wcs import WCS
from astropy.io import fits
import pyspeckit
from skimage.morphology import remove_small_objects, remove_small_holes, closing, disk, opening
import os
from scipy.interpolate import griddata
# import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

## Preparation: loading files and making mask

### Filenames

In [17]:
# filenames

fitdir = 'gaussfit/'
imagefile = hc3n_10_9_cube_s + '_K.fits'
rmsfile = hc3n_10_9_cube_s + '_rms.fits'
snrfile = hc3n_10_9_cube_s + '_K_-3.0_18.0_snr.fits'
maskfile = fitdir + 'HC3N_10_9_mask'

fitfile =  fitdir + 'HC3N_10_9_1G_fitparams.fits'
fitfilefiltered = fitdir + 'HC3N_10_9_1G_fitparams_filtered.fits'
newguessfile = fitdir + 'HC3N_10_9_1G_fitparams_filtered_guesses.fits'
fitfile2 = fitdir + 'HC3N_10_9_1G_fitparams_2.fits'
fitfile2filtered = fitdir + 'HC3N_10_9_1G_fitparams_2_filtered.fits'

snratio = 4
verbosity = False
minsize = 1 #in beam area
multicore = 40

### Functions

In [25]:
def filtersolutions(parcube, errcube, npeaks, rmsmap=None, snratio=None, 
                    velinit=-np.inf, velend=np.inf, filter_negative=False, 
                    errorfrac=None, eps=1.e-6, filter_islands=False, 
                    minsizetrim=50, chansize=0.08, masked_chans=False, 
                    velinit_mask=None, velend_mask=None, widthfilter=False):
    """
    Replace the pixels in the fitted cube with np.nan where the fit is not
    good enough according to our criteria.

    The criteria that a pixel must have are:
    - The errors are not zero (less than eps)
    - The peak must not be negative in case filter_negatives is true
    - The error fraction is lower than errorfrac, if given
    - The central velocity value must be within the range [velinit,velend]
    - The peak value must be larger than rms times snratio, if given
    - If one parameter in a spectra is np.nan, all the spectra must be nan (sanity
    check)
    - NEW 10.5.22: the uncertainty in the central velocity must be smaller than the channel width
    - NEW 11.5.22: if we masked certain channels, the central velocity of the components must not be within those channels

    Args:
        variable (type): description

    Returns:
        type: description

    Raises:
        Exception: description

    """
    # we first create all the masks we need
    
    # all errors must be non zero
    # note that eps must be larger than the velocity dispersion
    zeromask = np.zeros(np.shape(parcube[0]), dtype=int) # we need to do a plane mask
    for i in range(3*npeaks):
        zeromask += np.where(np.abs(errcube[i])<eps, 1, 0)
        
    # if a fraction is given, make sure all values have an error fraction less than that
    errormask = np.zeros(np.shape(parcube[0]), dtype=int)
    if errorfrac is not None:
        for i in range(3*npeaks):
            errormask += np.where(np.abs(errcube[i]/parcube[i]) > errorfrac, 1, 0)
            
    # if indicated, the velocity width must not be larger than the selection range
    widthmask = np.zeros(np.shape(zeromask), dtype=int)
    if widthfilter:
        for i in range(npeaks):
            widthmask += np.where(parcube[2+3*i] > (velend-velinit)/2.35, 1, 0)
        
    
    # if indicated, all values must be non-negative
    negativemask = np.zeros(np.shape(zeromask), dtype=int)
    if filter_negative:
        for i in range(3*npeaks):
            negativemask += np.where(parcube[i] < 0, 1, 0)
    
    # velocities must be within range
    velocitymask = np.zeros(np.shape(zeromask), dtype=int)
    for i in range(npeaks):
        velocitymask += np.where(parcube[1+3*i] < velinit, 1, 0) + \
        np.where(parcube[1+3*i] > velend, 1, 0)
    
    # the amplitude of the Gaussian must be above the snratio indicated
    peakmask = np.zeros(np.shape(zeromask), dtype=int)
    if snratio is not None and rmsmap is not None:
        for i in range(npeaks):
            snrmappeak = parcube[3*i] / rmsmap
            peakmask += np.where(snrmappeak < snratio, 1, 0)
    
    # all values of parameters and uncertainties must be not NaN
    nanmask = np.zeros(np.shape(zeromask), dtype=int)
    for i in range(3*npeaks):
        nanmask += np.where(np.isnan(parcube[i]), 1, 0) + np.where(np.isnan(errcube[i]), 1, 0)
    
    # central velocity uncertainty must be lower than
    centraluncertaintymask = np.zeros(np.shape(zeromask), dtype=int)
    for i in range(npeaks):
        velocitymask += np.where(errcube[1+3*i] > chansize, 1, 0)
        
    # if indicated, the central velocities must not be within masked channels
    maskedchansmask = np.zeros(np.shape(zeromask), dtype=int)
    if masked_chans:
        for i in range(npeaks):
            maskedchansmask += np.where(parcube[1+3*i] > velinit_mask, 1, 0) * np.where(parcube[1+3*i] < velend_mask, 1, 0)
    
    finalmask = zeromask + errormask + negativemask + widthmask + velocitymask + peakmask + nanmask + centraluncertaintymask + maskedchansmask
    
    parcubenew = parcube.copy()
    errcubenew = errcube.copy()
    parcubenew[np.where(np.repeat([finalmask], 3*npeaks, axis=0))] = np.nan
    errcubenew[np.where(np.repeat([finalmask], 3*npeaks, axis=0))] = np.nan   
    
    # eliminate isolated small islands of emission after the filter
    if filter_islands:        
        planemask = ~np.isnan(parcubenew[0])
        planemask = remove_small_objects(planemask, min_size=minsizetrim)
        smallmask = np.ones(np.shape(planemask), dtype=int) - planemask
        parcubenew[np.where(np.repeat([smallmask], 3*npeaks, axis=0))] = np.nan
        errcubenew[np.where(np.repeat([smallmask], 3*npeaks, axis=0))] = np.nan
    
    return parcubenew, errcubenew


# as each plane represents a different physical characteristic, and they do not necessarilly 
# correlate between each other, each plane of initguesses must be fit separately
def interpolatesolutions(solfilein, npeaks, mask=None):
    '''
    The solfilein must be a .fits file that contains one parameter per 
    plane and then one parameter uncertainty per plane.
    The shape must be [nplane, yy, xx]
    The mask must be 2 dimensional
    '''
    solcube = fits.getdata(solfilein)[:3*npeaks]
    if np.any(np.isnan(solcube)):
        solcube[np.where(np.isnan(solcube))] = 0
    solcubeshape = np.shape(solcube)
    yy, xx = np.indices(solcubeshape[1:])
    filledcube = solcube.copy()
    headersolcube = fits.getheader(solfilein)

    for i, plane in enumerate(solcube):
        indexknown = np.where(plane<1e-5, False, True)
        filledcube[i][~indexknown] = griddata((xx[indexknown], yy[indexknown]),
                                                  plane[indexknown],
                                                  (xx[~indexknown], yy[~indexknown])
                                                 )
        if mask is not None:
            filledcube[i][np.where(mask==0)] = np.nan
    return filledcube, headersolcube


### Create the mask

In [11]:
cube = pyspeckit.Cube(imagefile)
header = cube.header

beamarea, beamarea_pix2 = beam_size(header)
minsizetrim = minsize * beamarea_pix2
chansize = np.abs(header['CDELT3'])

rmsmap = fits.getdata(rmsfile)
snrmap, headerimage = fits.getdata(snrfile, header=True)



In [22]:
limitedmin=[True, True, True]
limitedmax=[False, True, True]
minpars=[0, velrange[0], 0]
maxpars=[0, velrange[1], velrange[1]-velrange[0]]

In [12]:
if not os.path.exists(maskfile+'.fits'):
    hdcube = headerimage.copy()
    hdcube['BUNIT'] = ''
    planemask = (snrmap > snratio)
    fits.writeto(maskfile+'_0.fits', planemask.astype(int), hdcube)
    # check the resulting mask map to see how much does the minimum size have to be and its connectivity
    # before applying this filter
    planemask = remove_small_objects(planemask, min_size=minsizetrim) # removes small islands of emission
    fits.writeto(maskfile+'_1.fits', planemask.astype(int), hdcube)
    planemask = remove_small_holes(planemask, area_threshold=minsizetrim) # fills small holes inside the emission area
    fits.writeto(maskfile+'_2.fits', planemask.astype(int), hdcube)
    planemask = closing(planemask, disk(1))
    planemask = remove_small_holes(planemask, area_threshold=6)
    fits.writeto(maskfile+'.fits', planemask.astype(int), hdcube)
    print('Created Mask file')
else:
    planemask = fits.getdata(maskfile+'.fits')
    print('Loaded Mask file')

Loaded Mask file


## One Gaussian fit

In [13]:
initguesses = [2, 8, 0.5]
starting_point = (194,264)


In [14]:
if not os.path.exists(fitfile):
    cube.fiteach(fittype='gaussian',
                 signal_cut=snratio,
                 guesses=initguesses,
                 errmap = rmsmap, 
                 maskmap = planemask,
                 limitedmin=limitedmin,
                 limitedmax=limitedmax,
                 minpars=minpars,
                 maxpars=maxpars,
                 use_neighbor_as_guess=True, 
                 start_from_point=starting_point,
                 verbose=verbosity,
                 multicore=multicore)
    cube.write_fit(fitfile)
    fittedmodel = cube.get_modelcube()
    
else:
    cube.load_model_fit(fitfile, 3, fittype='gaussian')

INFO: Left region selection unchanged.  xminpix, xmaxpix: 0,195 [pyspeckit.spectrum.interactive]
INFO: Left region selection unchanged.  xminpix, xmaxpix: 0,195 [pyspeckit.spectrum.interactive]


NaN or infinite values encountered in parameter cube.


### Quality assessment

In [15]:

if not os.path.exists(fitfilefiltered):
    print("Creating filtered version.") 
    parcube, errcube = filtersolutions(cube.parcube, cube.errcube, 1, 
                                       rmsmap=rmsmap, snratio=snratio, 
                                       velinit=velrange[0], velend=velrange[1], 
                                       filter_negative=True, errorfrac=0.5, 
                                       filter_islands=True, chansize=chansize)
    cube.parcube = parcube
    cube.errcube = errcube
    cube.write_fit(fitfilefiltered) #(fitfile2filtered)
    fittedmodel = cube.get_modelcube()
    
else:
    print("Loading filtered version.")
    cube.load_model_fit(fitfilefiltered, 3, fittype='gaussian')
    fittedmodel = cube.get_modelcube()

Creating filtered version.


invalid value encountered in divide


In [18]:
if not os.path.exists(newguessfile):
    print("Interpolating previous solutions.")
    newinitguess, headerguess = interpolatesolutions(fitfilefiltered, 1, mask=planemask)
    fits.writeto(newguessfile, newinitguess, headerguess)
    
else:
    print("Interpolation exists. Loading.")
    newinitguess = fits.getdata(newguessfile)

Interpolating previous solutions.


### Second fit

In [23]:
if not os.path.exists(fitfile2):
    print("Starting fit")
    cube.fiteach(fittype='gaussian',
                 guesses=newinitguess,
                 errmap = rmsmap, 
                 maskmap = planemask,
                 limitedmin=limitedmin,
                 limitedmax=limitedmax,
                 minpars=minpars,
                 maxpars=maxpars,
                 use_neighbor_as_guess=True, 
                 start_from_point=starting_point,
                 verbose=verbosity,
                 verbose_level=1,
                 multicore=multicore)
    cube.write_fit(fitfile2)
    fittedmodel = cube.get_modelcube()
    
else:
    print("Fit 2 exists. Loading")
    cube.load_model_fit(fitfile2, 3, fittype='gaussian')

Starting fit
INFO: Fitting up to 50709 spectra [pyspeckit.cubes.SpectralCube]
INFO: Left region selection unchanged.  xminpix, xmaxpix: 0,195 [pyspeckit.spectrum.interactive]


Mean of empty slice
Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [26]:
if not os.path.exists(fitfile2filtered):
    print("Creating filtered version.") 
    parcube, errcube = filtersolutions(cube.parcube, cube.errcube, 1, 
                                       rmsmap=rmsmap, snratio=snratio, 
                                       velinit=velrange[0], velend=velrange[1], 
                                       filter_negative=True, errorfrac=0.5, 
                                       filter_islands=True, chansize=chansize, widthfilter=True)
    cube.parcube = parcube
    cube.errcube = errcube
    cube.write_fit(fitfile2filtered) #(fitfile2filtered)
    fittedmodel = cube.get_modelcube()
    
else:
    print("Loading filtered version.")
    cube.load_model_fit(fitfile2filtered, 3, fittype='gaussian')
    fittedmodel = cube.get_modelcube()

Creating filtered version.
